# 1. Liste des élus romands

In [1]:
import requests
import pandas as pd

In [2]:
members_response = requests.get('http://ws-old.parlament.ch/councillors/basicdetails?format=json', headers={'Accept': 'application/json'})
members = members_response.json()

In [3]:
# liste des elus
df = pd.DataFrame(members)
df.shape

(245, 16)

In [4]:
# romands seulement
df = df[df['canton'].isin(['VD', 'GE', 'FR', 'JU', 'VS', 'NE'])].copy()
df.shape

(62, 16)

# 2. Liste des interventions (test)

In [5]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
binary = FirefoxBinary('/Applications/Firefox_32.app/Contents/MacOS/firefox-bin')
driver = webdriver.Chrome(r'_helpers/chromedriver')
from time import sleep
import re

## On va prendre les Jurassiens

In [6]:
df_ju = df[df['canton'] == 'JU']

In [7]:
test_id = df_ju['id'].values[0]

In [8]:
member_speeches = 'https://www.parlament.ch/fr/ratsbetrieb/suche-Amtliches-bulletin#Default=%7B%22k%22:%22PdSubjectSpeakerPersonNumbers:{member_id}%22%7D'

In [9]:
driver.get(member_speeches.format(member_id=test_id))

In [10]:
rows = []
continueScraping = True

In [11]:
def getSpeeches():
    search_group = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#Groups .ms-srch-group'))
    )
    
    if not search_group:
        return False
    
    # On parcourt l’innerHTML avec BeautifulSoup
    doc = BeautifulSoup(search_group.get_attribute('innerHTML'), 'html.parser')
    
    results = doc.select('.ms-srch-item')

    current_rows = []
    for result in results:
        link = 'https://www.parlament.ch' + result.find('a').get('href')
        label = result.find('h4').text.strip()
        no = result.find('div', {'class': 'ms-srch-item-excerpt'}).find('span').string
        title = result.find('div', {'class': 'ms-srch-item-excerpt'}).find_all('span')[1].string
        current_rows.append([no, label, title, link])
    print(len(current_rows), 'rows found.')
    return current_rows

In [12]:
while continueScraping:
    scraping_result = getSpeeches()
    if scraping_result:
        rows.extend(scraping_result)
        #dfi = pd.DataFrame(rows, columns=['no', 'label', 'title', 'link'])
        last_result_year = re.match('.*\.(\d{4}).*', scraping_result[-1][1]).group(1)
        if int(last_result_year) < 2015:
            print('Before 2014, so we stop')
            continueScraping = False
        
        # prochaine page?
        nextPageLink = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '#PageLinkNext'))
            )
        if nextPageLink:
            nextPageLink.click()
            sleep(5)
            getSpeeches()
    else:
        print('Last page, so we stop')
        continueScraping = False

10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.
10 rows found.


In [16]:
df_member = pd.DataFrame(rows, columns=['no', 'label', 'title', 'href'])

In [22]:
# a regarder de plus pres
df_member.drop_duplicates(subset=['no']).shape

(86, 4)

In [23]:
df_member.to_csv('data/speech_list/{}.csv'.format(test_id))

# 3. Scraping des interventions (test)

In [48]:
speech_href = df_member['href'].values[1]

In [49]:
driver.get(speech_href)

In [30]:
# Problème: l’intervention peut être noyée parmi celles des autres, non?
# On pourrait récupérer les résumés au stade 2 et voir si ça matche

In [ ]:
# Plus simple: on merge les duplicates, et on checke avec l’id de l’élu

In [31]:
transcripts = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, '#areaDebates'))
)

In [32]:
doc = BeautifulSoup(transcripts.get_attribute('innerHTML'), 'html.parser')

In [82]:
speeches = doc.select('.pd-person-description')

In [94]:
# on fera un iterrow() sur le df des élus
row = df_ju.iloc[0]
memberId = row['biographyUrl'].split('=')[-1]

In [101]:
member_speeches = []
for speech in speeches:
    member_href = speech.find('a', {'class': 'person-name'})
    if member_href:
        print('Nom trouvé:', member_href.text.strip())
    
        if member_href.get('href'):
            if member_href.get('href').split('=')[-1] == memberId:
                print('match avec le no de l’élu:', memberId)
                if len(speech.find_all('p')) > 1:
                    print('merde, il peut y avoir > 1 paragraphe => à revoir')
                member_speeches.append(speech.find('p').text)
            else:
                print('Pas de match')
        else:
            print('Pas de lien sur le nom:', member_href.string.strip())

Nom trouvé: Fridez Pierre-Alain
match avec le no de l’élu: 4074
Nom trouvé: Amherd Viola
Pas de lien sur le nom: Amherd Viola


In [105]:
# a merger avec bio des élus
df_speeches = pd.DataFrame(member_speeches, columns=['Texte ou intervention'])
df_speeches.to_csv('data/{}_{}.csv'.format(test_id, row['lastName']), index=False)